# Neural Horizon MPC tutorial

this script will illustrate the generation and usage of the Neural Horizon MPC

In [ ]:
import os
import torch
import numpy as np

from itertools import product
from tqdm.notebook import tqdm_notebook
from pprint import pprint 
from copy import deepcopy

In [ ]:
from src.inverted_pendulum import *
from src.mpc_classes import *
from src.neural_horizon import NN_for_casadi, load_NN
from src.data_generation import MPC_dataset_gen
from src.parameters import MPC_Param, NH_AMPC_Param
from src.plotting import plot_MPC_results
from src.utils import save_results

## Set initial parameter values

In [ ]:
RESULTS_DIR = os.path.abspath('Results')
MPC_DATASETS_DIR = os.path.join(RESULTS_DIR, 'MPC_data_gen')
NNS_DIR = os.path.join(RESULTS_DIR, 'Trained_Networks')
MPC_RESULTS_DIR = os.path.join(RESULTS_DIR, 'NH_CMPC_results')

MAKE_TEST_RESULTS = True

if MAKE_TEST_RESULTS:
    MPC_RESULTS_DIR += '_Test'

NUM_SAMPLES = 30_000

NUM_NNS = 10
USE_CUDA = False

MPC_PARAM_DICT = {
    'T_sim': 5, # length of the closed-loop simulation (in seconds)
}

NH_CMPC_OPTIONS = [tup for tup in product(
    (8, ),                                      # N_MPC
    (22, ),       # N_NN -> if USE_BEGINING_OF_DATASET != 'begin, it has to be 70-max(N_MPCs)
    (70, ),                                     # N_DATASET
    (5, ),                                      # TRAIN_DATASET_VERSION
    (6, ),                                      # TEST_DATASET_VERSION
    ('fixed', ),                                # USE_BEGINING_OF_DATASET ('begin', 'fixed', '') if '' use DS_FEATURE of N_MPC
    (8, ),                                      # DS_FEATURE_IF_FIXED
    ('RTI_PCHPIPM_DISCRETE', ),                 # DATASET_NAME_ADD ('RTI_PCHPIPM_DISCRETE', 'RTI_PCHPIPM_DISCRETE_50ITER', 'RTI_PCHPIPM_ROBUST_DISCRETE')
    (32, ),                                     # HIDDEN_NEURONS
    (None, ),                           # END_HIDDEN_SIZES (left from pruning)
)]

In [ ]:
NH_CMPC_PARAMS = [
    NH_AMPC_Param(
        # Param
        N_MPC = N_MPC, 
        N = N_NN+N_MPC,

        # Dataset stuff
        N_DS = N_DSET, 
        TRAIN_V_DS = TRAIN_DATASET_VERSION, 
        TEST_V_DS = TEST_DATASET_VERSION, 
        DS_begin = USE_BEGIN,
        DS_samples = NUM_SAMPLES,
        DS_opts_name = DS_OPT_NAME,
        DS_feature = DS_FEATURES,

        # NN stuff
        V_NN = NN_VERSION,
        N_hidden = N_HIDDEN,
        N_hidden_end = END_N_SIZES,

        # Param
        **MPC_PARAM_DICT
    ) for NN_VERSION in range(NUM_NNS) \
        for N_MPC, N_NN, N_DSET, TRAIN_DATASET_VERSION, TEST_DATASET_VERSION, USE_BEGIN,\
            DS_FEATURES, DS_OPT_NAME, N_HIDDEN, END_N_SIZES in NH_CMPC_OPTIONS \
]

## Loading the data and trained Neural Networks

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
dtype = torch.float32
print(device)

## Generating closed-loop trajectories for baseline and Neural horizon MPC

In [ ]:
for nh_cmpc_params in NH_CMPC_PARAMS:
    results_filename = f'NH_CMPC_results_{nh_cmpc_params.Pruned_NN_name if nh_cmpc_params.N_hidden_end is not None else nh_cmpc_params.NN_name}'

    NN_fc = load_NN(nh_cmpc_params, NNS_DIR, MPC_DATASETS_DIR, device, dtype)
    NN_fc.evaluate_NN(os.path.join(MPC_DATASETS_DIR, nh_cmpc_params.test_DS_name))

    cMPC = MPC_NN_class(NN_fc)
    Fmodel = F_model_RK4(nonlinear_pend_dynamics(cMPC.P),cMPC.P)
    results = get_MPC_trajectory(cMPC,Fmodel)

    save_results(os.path.join(MPC_RESULTS_DIR, results_filename), results, always_overwrite=True)